Antes de cualquier cosa, debes autentificarte a través del cmd, ejecutando:<br>
earthengine authenticate <br>
y antes deberías instalar pip install earthengine-api geemap geopandas pandas

In [1]:
import ee
import geopandas as gpd
import pandas as pd
from datetime import datetime
from shapely.geometry import mapping
import json
from tqdm import tqdm
import csv

Hay que ir a 
https://console.cloud.google.com/apis/dashboard?invt=AbuKWg&project=maximal-osprey-287122
para habilitar la consulta.

In [2]:
# Inicializar Earth Engine
ee.Initialize(project='maximal-osprey-287122')  # Reemplaza con tu ID real

In [3]:
shapefile_path = "C:/Users/Michael/Documents/ml_zc/ecu_adm_2024/ecu_adm_adm3_2024.shp"
gdf = gpd.read_file(shapefile_path)

In [4]:
parroquias = gdf[['ADM3_PCODE', 'geometry']]
parroquias[parroquias['ADM3_PCODE'] == 'EC160156']
#parroquias = parroquias[parroquias['ADM3_PCODE'].isin(['EC050651', 'EC010951'])] #'EC050651', 
parroquias = parroquias.iloc[774:]

In [5]:
# Función para convertir a EE FeatureCollection correctamente
def gdf_to_fc(tabla_geometria):
    features = []
    for _, row in tabla_geometria.iterrows():
        # Convierte la geometría a formato geojson (dict) con mapping()
        geojson_geom = mapping(row['geometry'])
        geom = ee.Geometry(geojson_geom)
        props = row.drop('geometry').to_dict()
        features.append(ee.Feature(geom, props))
    return ee.FeatureCollection(features)

# El NDVI se calcula utilizando las bandas de infrarrojo cercano (B8) y rojo (B4) de Sentinel-2:
def calcular_ndvi(image):
    # Calcular NDVI
    ndvi = image.normalizedDifference(['B8', 'B4']).rename('NDVI')
    return ndvi

In [6]:
# Definir años a procesar
#years = [a for a in range(2015, 2025)]
years = [2022]

# Abrir el archivo en modo escritura (o append si ya existe)
with open('C:/Users/Michael/Documents/ml_zc/DatosObtenidos/nvdi_22_2.csv', mode='w', newline='', encoding='utf-8') as file:
    # Crear el escritor CSV
    writer = csv.DictWriter(file, fieldnames=['ADM3_PCODE', 'Año', 'avg_ndvi'])
    
    # Escribir el encabezado (si es necesario)
    writer.writeheader()
    
    for num_fila in tqdm(range(0, parroquias.shape[0]), desc="Procesando parroquias", ncols=parroquias.shape[0]):
        
        # Obtener fila de la parroquia
        parroquia = parroquias.iloc[num_fila]
    
        # Convertir la fila (Serie) a un GeoDataFrame
        temp_gdf = gpd.GeoDataFrame([parroquia], geometry='geometry', crs=parroquias.crs)
    
        # Transformamos el objeto a formato compatible con Google Earth Engine
        polygon = gdf_to_fc(temp_gdf)

        # Extraemos el código de la parroquia para usarla luego
        name_parr = parroquia['ADM3_PCODE']

        # Mostrar parroquia en proceso
        print(name_parr)

        # Extraer VIIRS diciembre por parroquia:

        # Iterar sobre el periodo
        for year in years:
            start = ee.Date.fromYMD(year, 1, 1)  # Comienza desde el 1 de enero
            end = ee.Date.fromYMD(year, 12, 31)   # Termina el 31 de diciembre
            
            # Son datos 2015 - 2025.  Ver en: https://developers.google.com/earth-engine/datasets/catalog/COPERNICUS_S2?hl=es-419
            image_collection = ee.ImageCollection('COPERNICUS/S2_HARMONIZED').filterBounds(polygon)\
                    .filterDate(start, end)\
                    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20))
    
            # Verificar cuántas imágenes fueron seleccionadas
            num_images = image_collection.size().getInfo()
            print(f"     Imágenes seleccionadas para {year}: {num_images}")

            if num_images > 0:
                # Aplicar la función para calcular el NDVI a todas las imágenes
                ndvi_collection = image_collection.map(calcular_ndvi)

                # Calcular la media de NDVI para todo el periodo
                ndvi_mean = ndvi_collection.mean()

                # Reducir el NDVI por el polígono (media dentro del polígono)
                ndvi_zonal = ndvi_mean.reduceRegion(
                    reducer=ee.Reducer.mean(),
                    geometry=polygon,
                    scale=10,  # Resolución espacial de Sentinel-2
                    maxPixels=1e8
                )

                # Verificar el contenido del resultado
                ndvi_zonal_info = ndvi_zonal.getInfo()
                print(f"     Resultado de NDVI para {year}: {ndvi_zonal_info}")

                if ndvi_zonal_info:
                    writer.writerow({'ADM3_PCODE': name_parr, 'Año': year, 'avg_ndvi': ndvi_zonal_info['NDVI']})
                else:
                    print(f"       >> No se obtuvieron valores de NDVI para {year}")
            else:
                print(f"     >>No se encontraron imágenes para el año {year}")

Procesando parroquias:   0%|                                                                                                                                                                                                                        | 0/268 [00:00<?, ?it/s]

EC160156


Procesando parroquias:   0%|                                                                                                                                                                                                                        | 0/268 [00:18<?, ?it/s]


EEException: Request payload size exceeds the limit: 10485760 bytes.